In [12]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')
import requests
from bs4 import BeautifulSoup

In [386]:
URL = 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051_indicators.txt'
Variables = ['Rank', 'Country' ,'Region', 'Name', 'Faculty Total', 'Faculty International','Students Total', 'Students International','Student to Faculty','Ratio International']
QS = pd.DataFrame(columns=Variables)
QS['Rank'] = QS['Rank'].astype(np.int64)
QS['Faculty Total'] = QS['Faculty Total'].astype(np.int64)
QS['Faculty International'] = QS['Faculty International'].astype(np.int64)
QS['Students Total'] = QS['Students Total'].astype(np.int64)
QS['Students International'] = QS['Students International'].astype(np.int64)
QS['Student to Faculty'] = QS['Student to Faculty'].astype(np.int64)
QS['Ratio International'] = QS['Ratio International'].astype(np.int64)

r = requests.get(URL)
for i in range(10):
    university = r.json()['data'][i]['uni']
    soup = BeautifulSoup(university, 'html.parser')
    name = soup.get_text()
    rank = r.json()['data'][i]['overall_rank']
    region = r.json()['data'][i]['region']
    country = r.json()['data'][i]['location']
    faculty_total = np.nan
    faculty_inter = np.nan
    students_total = np.nan
    students_inter = np.nan

    #request URL deeper level
    URL2 = 'https://www.topuniversities.com' + soup.find('a')['href']
    r2 = requests.get(URL2)
    soup2 = BeautifulSoup(r2.text, 'html.parser')
    #extract numbers from deeper level
    if(len(soup2.find_all('div',class_='number')) != 0):
        faculty_total = int(soup2.find_all('div',class_='number')[0].get_text().strip().replace(',',''))
        faculty_inter = int(soup2.find_all('div',class_='number')[1].get_text().strip().replace(',',''))
        students_total = int(soup2.find_all('div',class_='number')[2].get_text().strip().replace(',',''))
        students_inter = int(soup2.find_all('div',class_='number')[4].get_text().strip().replace(',',''))
    student_to_faculty = students_total/faculty_total
    ratio_inter = students_total/students_inter
    row = pd.DataFrame({'Ratio International':ratio_inter,'Student to Faculty':student_to_faculty,'Region': region, 'Country': country, 'Name': name, 'Rank': [rank],'Faculty Total':[faculty_total],'Faculty International':[faculty_inter],'Students Total':[students_total],'Students International':[students_inter]})
    QS = QS.append(row,ignore_index=True)
QS = QS.reindex_axis(Variables,axis=1)
QS.set_index('Name')



,Rank,Country,Region,Faculty Total,Faculty International,Students Total,Students International,Student to Faculty,Ratio International
Name,,,,,,,,,
Massachusetts Institute of Technology (MIT),1,United States,North America,2982,1679,11067,2982,3.711268,3.711268
Stanford University,2,United States,North America,4285,2042,15878,4285,3.705484,3.705484
Harvard University,3,United States,North America,4350,1311,22429,4350,5.156092,5.156092
California Institute of Technology (Caltech),4,United States,North America,953,350,2255,953,2.366212,2.366212
University of Cambridge,5,United Kingdom,Europe,5490,2278,18770,5490,3.418944,3.418944
University of Oxford,6,United Kingdom,Europe,6750,2964,19720,6750,2.921481,2.921481
UCL (University College London),7,United Kingdom,Europe,6345,2554,31080,6345,4.898345,4.898345
Imperial College London,8,United Kingdom,Europe,3930,2071,16090,3930,4.094148,4.094148
University of Chicago,9,United States,North America,2449,635,13557,2449,5.535729,5.535729


In [389]:
QS.groupby(['Country']).agg({'Ratio International': np.mean, 'Student to Faculty':np.mean})

,,Ratio International,Student to Faculty
Country,Region,,
Switzerland,Europe,7.999596,7.999596
United Kingdom,Europe,3.833229,3.833229
United States,North America,4.094957,4.094957


In [390]:
QS.groupby(['Region']).agg({'Ratio International': np.mean, 'Student to Faculty':np.mean})

,Ratio International,Student to Faculty
Region,,
Europe,4.666503,4.666503
North America,4.094957,4.094957


In [409]:
#check my calculations for number of international students and number of faculty members

URL = 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json'
Variables = ['Rank', 'Country' ,'Region', 'Name', 'Faculty Total', 'Faculty International','Students Total', 'Students International','Student to Faculty','Ratio International']
THE = pd.DataFrame(columns=Variables)
THE['Rank'] = THE['Rank'].astype(np.int64)
THE['Faculty Total'] = THE['Faculty Total'].astype(np.int64)
THE['Faculty International'] = THE['Faculty International'].astype(np.int64)
THE['Students Total'] = THE['Students Total'].astype(np.int64)
THE['Students International'] = THE['Students International'].astype(np.int64)
THE['Student to Faculty'] = THE['Student to Faculty'].astype(np.int64)
THE['Ratio International'] = THE['Ratio International'].astype(np.int64)

r = requests.get(URL)
for i in range(10):
    name = r.json()['data'][i]['name']
    rank = int(r.json()['data'][i]['rank'].replace('=',''))
    country = r.json()['data'][i]['location']
    region = np.nan
    if len(QS.loc[QS['Country'] == country]['Region']) != 0:
        region = QS.loc[QS['Country'] == country]['Region'].iloc[0]
    students_per_staff = float(r.json()['data'][i]['stats_student_staff_ratio'])
    students_total = float(r.json()['data'][i]['stats_number_students'].strip().replace(',',''))
    faculty_total = students_total/students_per_staff
    ratio_inter = 100/int(r.json()['data'][i]['stats_pc_intl_students'].replace('%',''))
    students_inter = students_total/ratio_inter
    row = pd.DataFrame({'Faculty Total': faculty_total,'Students International':students_inter,'Region':region,'Country': country, 'Name': name, 'Rank': [rank],'Student to Faculty':[students_per_staff],'Students Total':[students_total],'Percentage of international students':[percentage_inter_students],'Ratio International':ratio_inter})
    THE = THE.append(row,ignore_index=True)
THE = THE.reindex_axis(Variables,axis=1)
THE.set_index('Name')



,Rank,Country,Region,Faculty Total,Faculty International,Students Total,Students International,Student to Faculty,Ratio International
Name,,,,,,,,,
University of Oxford,1,United Kingdom,Europe,1822.232143,NaN,20409.0,7755.42,11.2,2.631579
University of Cambridge,2,United Kingdom,Europe,1687.064220,NaN,18389.0,6436.15,10.9,2.857143
California Institute of Technology,3,United States,North America,339.846154,NaN,2209.0,596.43,6.5,3.703704
Stanford University,3,United States,North America,2112.666667,NaN,15845.0,3485.90,7.5,4.545455
Massachusetts Institute of Technology,5,United States,North America,1284.712644,NaN,11177.0,3800.18,8.7,2.941176
Harvard University,6,United States,North America,2283.820225,NaN,20326.0,5284.76,8.9,3.846154
Princeton University,7,United States,North America,958.433735,NaN,7955.0,1909.20,8.3,4.166667
Imperial College London,8,United Kingdom,Europe,1390.964912,NaN,15857.0,8721.35,11.4,1.818182
University of Chicago,9,United States,North America,2181.451613,NaN,13525.0,3381.25,6.2,4.000000


In [400]:
THE.groupby(['Country']).agg({'Ratio International': np.mean, 'Student to Faculty':np.mean})

,Ratio International,Student to Faculty
Country,,
Switzerland,2.631579,14.600000
United Kingdom,2.435635,11.166667
United States,3.867193,7.683333


In [401]:
QS.groupby(['Region']).agg({'Ratio International': np.mean, 'Student to Faculty':np.mean})

,Ratio International,Student to Faculty
Region,,
Europe,4.666503,4.666503
North America,4.094957,4.094957
